In [1]:
import cv2
import numpy as np
import os
import glob
import sys # 导入 sys 来修改路径

# --- 关键的路径设置 ---
# 告诉 Python 在哪里可以找到 utils 文件夹
# ../ 是返回上一级 (即 COMP9517 根目录)
utils_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if utils_path not in sys.path:
    sys.path.append(utils_path)

# 现在我们可以安全地从 utils 导入了
try:
    from utils.data_loader import load_data # 假设有这个函数
    print("data_loader imported successfully.")
except ImportError:
    print("Could not import data_loader. Will load images manually.")
    load_data = None # 设置一个标记

# 打印一下，确保导入成功
print(f"OpenCV version: {cv2.__version__}")
print("Numpy, os, glob, and sys imported successfully.")

Could not import data_loader. Will load images manually.
OpenCV version: 4.12.0
Numpy, os, glob, and sys imported successfully.


In [2]:
# 1. 定义你的数据集的根目录
#    (这个相对路径现在是正确的，从 features/ notebook 出发)
DATASET_DIR = '../../data/AgroPest-12/'

# 2. 定义训练集图像的路径
TRAIN_IMAGE_DIR = os.path.join(DATASET_DIR, 'train', 'images')

# 3. 检查路径是否存在
if not os.path.exists(TRAIN_IMAGE_DIR):
    print(f"!!! 错误：找不到路径 !!!")
    print(f"请检查这个路径是否正确: {os.path.abspath(TRAIN_IMAGE_DIR)}")
else:
    print(f"成功找到数据文件夹: {os.path.abspath(TRAIN_IMAGE_DIR)}")

# 4. 获取所有训练图像的文件路径
image_paths = glob.glob(os.path.join(TRAIN_IMAGE_DIR, '*.jpg'))
image_paths.extend(glob.glob(os.path.join(TRAIN_IMAGE_DIR, '*.png')))

num_images = len(image_paths)
if num_images == 0:
    print("!!! 警告：在文件夹中没有找到 .jpg 或 .png 图像。请检查你的数据。")
else:
    print(f"Total training images found: {num_images}")

成功找到数据文件夹: E:\comp9517_project\data\AgroPest-12\train\images
Total training images found: 11502


In [3]:
# 1. 创建 SIFT 对象
sift = cv2.SIFT_create() 
print("SIFT detector created.")

# 2. 创建一个空列表，用来存放所有图像的 SIFT 描述符
all_descriptors_list = []

print("Starting SIFT feature extraction... (这可能需要几分钟)")

# 3. 遍历你找到的每一张图像路径
for i, img_path in enumerate(image_paths):
    
    # 4. 读取图像
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Warning: Could not read image {img_path}. Skipping.")
        continue
    
    # 5. 转换为灰度图
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # [cite_start]6. 检测关键点并计算描述符 (descriptors) [cite: 6546-6547, 6554]
    kp, des = sift.detectAndCompute(gray_img, None)
    
    # 7. 检查是否找到了特征
    if des is not None:
        # 8. 如果找到了，把这张图的描述符添加到我们的总列表中
        all_descriptors_list.append(des)
        
    # 打印进度
    if (i + 1) % 100 == 0 or (i + 1) == num_images:
        print(f"Processed {i + 1}/{num_images} images...")

print("...Feature extraction complete.")
print(f"Total images with features: {len(all_descriptors_list)}")

SIFT detector created.
Starting SIFT feature extraction... (这可能需要几分钟)
Processed 100/11502 images...
Processed 200/11502 images...
Processed 300/11502 images...
Processed 400/11502 images...
Processed 500/11502 images...
Processed 600/11502 images...
Processed 700/11502 images...
Processed 800/11502 images...
Processed 900/11502 images...
Processed 1000/11502 images...
Processed 1100/11502 images...
Processed 1200/11502 images...
Processed 1300/11502 images...
Processed 1400/11502 images...
Processed 1500/11502 images...
Processed 1600/11502 images...
Processed 1700/11502 images...
Processed 1800/11502 images...
Processed 1900/11502 images...
Processed 2000/11502 images...
Processed 2100/11502 images...
Processed 2200/11502 images...
Processed 2300/11502 images...
Processed 2400/11502 images...
Processed 2500/11502 images...
Processed 2600/11502 images...
Processed 2700/11502 images...
Processed 2800/11502 images...
Processed 2900/11502 images...
Processed 3000/11502 images...
Processed

In [4]:
# 1. 使用 numpy 的 vstack (vertical stack) 功能将所有描述符数组垂直堆叠
print("Consolidating all descriptors into one array...")
all_descriptors = np.vstack(all_descriptors_list)

print(f"Shape of final descriptors array: {all_descriptors.shape}")
print(f"(This means {all_descriptors.shape[0]} total features found across all images)")

# 2. 定义一个输出路径，用来保存这个大文件
#    我们把它保存在 'models' 文件夹里
OUTPUT_DIR = '../models/' # 返回上一级，进入 models 文件夹
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR) # 如果 models 文件夹不存在，就创建它

OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'sift_all_train_descriptors.npy') 

# 3. 使用 numpy.save 保存
np.save(OUTPUT_FILE, all_descriptors)

print(f"All SIFT descriptors saved to: {os.path.abspath(OUTPUT_FILE)}")
print("=======================")
print("✅ Task 1.1 COMPLETED.")
print("=======================")
print(f"\n下一步: 把 {OUTPUT_FILE} 这个文件发给 William，他要用它来跑 K-Means。")

Consolidating all descriptors into one array...
Shape of final descriptors array: (15913117, 128)
(This means 15913117 total features found across all images)
All SIFT descriptors saved to: E:\comp9517_project\COMP9517\models\sift_all_train_descriptors.npy
✅ Task 1.1 COMPLETED.

下一步: 把 ../models/sift_all_train_descriptors.npy 这个文件发给 William，他要用它来跑 K-Means。
